In [ ]:
import pandas as pd
import requests
import numpy as np
from bs4 import BeautifulSoup
import re

In [ ]:
def get_director_genre_production(link):
    dict_data = {}

    global session
    content = session.get(link)

    table_soup = BeautifulSoup(content.content, "html.parser")

    tables = table_soup.find("div", attrs={"id": "summary"})
    print(link)
    rows = (tables.find_all("table"))
    director_name = None
    try:
        try:
            cast_crew = link.replace("=summary", "=cast-and-crew")
            cast_tables = session.get(cast_crew)

            cast_soup = BeautifulSoup(cast_tables.content, "html.parser")

            cast_div = cast_soup.find_all("div", {"class": "cast_new"})
            cast_table = cast_div[2].find("table")
            director = cast_table.find_all("tr")[0].text.split("\n")
            index = director.index("Director")
            director_name = director[index - 2]
        except ValueError:
            director_name = None

        for tr in rows[4].find_all("tr"):
            td = tr.text.split("\n")
            if any(["Genre" in data for data in td]):
                dict_data["Genre"] = " ".join(td).split(":")[-1]
            elif any(["Production Companies" in data for data in td]):
                dict_data["Production Company"] = " ".join(td).split(":")[-1]
    except IndexError:
        return {
            "Production Company": None,
            "Genre": None,
            "Movie name": get_movie_name(link),
            "Director name": director_name
        }

        # elif any([""])
    dict_data.update({"Movie name": get_movie_name(link)})
    dict_data.update({"Director name": director_name})
    return dict_data


In [ ]:
def get_movie_name(link):
    pattern = re.compile("/[A-Za-z0-9-()]+#")
    print(link)
    x = re.search(pattern, link)
    if x is not None:
        start = x.start() + 1
        stop = x.end() - 1

        return link[start:stop]

In [ ]:
import concurrent.futures
def process_parallel(links):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        data = executor.map(get_director_genre_production, links)

    return data

In [ ]:
url_range = list(range(101, 1000, 100))
# print(url_range)
REQUIRED_ROWS = 100

session = requests.Session()
# print(soup)

dataframes = []
count = 1
hrefs = []
genre_director_company = []

while count < REQUIRED_ROWS:
    links = []
    url = 'https://www.the-numbers.com/box-office-records/worldwide/all-movies/cumulative/all-time/' + str(count)
    print(url)

    response = session.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    for item in soup.find_all('tr'):
        link = item.find_all('a')
        links.append(link)

        if len(link) > 1:
            hrefs.append("https://www.the-numbers.com" + link[-1]["href"])

    list_check = []
    for tr in soup.find_all('tr'):
        list_check.append(tr.get_text().split('\n'))

    array = np.array(list_check)
    array = array.reshape(101, 8)
    # print(np.size(array))

    df = pd.DataFrame(array, index=array[:, 1])
    new_header = df.iloc[0]
    df = df[1:]
    df.columns = new_header
    df.columns = (
        ['Rank', 'Rank', 'Year', 'Movie', 'WorldwideBox_Office', 'DomesticBox_Office', 'InternationalBox_Office',
         'Extra'])
    dataframes.append(df)
    # print(len(df))

    count += 100

# print(links[1][1])
new_df = pd.concat(dataframes)
new_df.to_csv("test_data.csv", index=False)

genre_director_company = process_parallel(hrefs)
other_details_df = pd.DataFrame(genre_director_company)
other_details_df.to_csv("director_genre_company.csv")
